In [1]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('structed-rdd').setMaster('local[*]')

sc = SparkContext(conf = conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 17:08:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc

<SparkContext master=local[*] appName=structed-rdd>

In [3]:
rdd = sc.textFile('file:/config/workspace/StructuredFile/dataset/kddcup.data')
rdd.getNumPartitions()

23

In [4]:
rdd = rdd.coalesce(10)
print(sc.defaultParallelism)
print(rdd.getNumPartitions())
rdd.persist()

64
10


CoalescedRDD[2] at coalesce at NativeMethodAccessorImpl.java:0

In [5]:
sc.setLogLevel('ERROR')

In [6]:
# Get 10 records randomly
rdd.takeSample(False, 10)

['0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,tcp,ftp_data,SF,172,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,0.00,0.00,0.00,0.00,1.00,0.00,0.00,166,81,0.45,0.04,0.45,0.02,0.01,0.00,0.01,0.00,normal.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,255,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,smurf.',
 '0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,131,8,1.00,1.00,0.00,0.00,0.06,0.07,0.00,255,8,0.03,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune.',
 '0,tcp,http,SF,328,1238,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,209,255,1.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,normal.',
 '0,icmp,ecr_i,SF,1032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,511,511,0.0

In [7]:
# get total number of records
rdd.count()

4898431

In [9]:
# calculate the ration of normal connections
normal_rdd = rdd.filter(lambda line: 'normal.' in line)

ratio = normal_rdd.count() / rdd.count()
print(ratio)

0.19859032412623553


In [10]:
# get the list of labels (last element of the csv)

rdd.map(lambda x: x.split(',')[-1]).distinct().collect()

['loadmodule.',
 'neptune.',
 'pod.',
 'warezclient.',
 'smurf.',
 'nmap.',
 'spy.',
 'back.',
 'teardrop.',
 'ipsweep.',
 'ftp_write.',
 'phf.',
 'multihop.',
 'guess_passwd.',
 'normal.',
 'land.',
 'imap.',
 'satan.',
 'portsweep.',
 'warezmaster.',
 'rootkit.',
 'buffer_overflow.',
 'perl.']

In [15]:
# count the number of connections for each label

counts = rdd.map(lambda x: (x.split(',')[-1], 1) ).reduceByKey(lambda x,y: x+y).collect()

In [16]:
counts

[('loadmodule.', 9),
 ('neptune.', 1072017),
 ('pod.', 264),
 ('warezclient.', 1020),
 ('smurf.', 2807886),
 ('nmap.', 2316),
 ('spy.', 2),
 ('back.', 2203),
 ('teardrop.', 979),
 ('ipsweep.', 12481),
 ('ftp_write.', 8),
 ('phf.', 4),
 ('multihop.', 7),
 ('guess_passwd.', 53),
 ('normal.', 972781),
 ('land.', 21),
 ('imap.', 12),
 ('satan.', 15892),
 ('portsweep.', 10413),
 ('warezmaster.', 20),
 ('rootkit.', 10),
 ('buffer_overflow.', 30),
 ('perl.', 3)]

In [18]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 70.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 43.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd

keys = [x[0] for x in counts]
values = [x[1] for x in counts]
pd_df = pd.DataFrame({
    'Label': keys,
    'Count': values
})
pd_df = pd_df.sort_values(by='Count', ascending=False)

In [24]:
pd_df

,Label,Count
4,smurf.,2807886
1,neptune.,1072017
14,normal.,972781
17,satan.,15892
9,ipsweep.,12481
18,portsweep.,10413
5,nmap.,2316
7,back.,2203
3,warezclient.,1020
8,teardrop.,979


In [ ]:
# Get the connection type with successful root_shell (item 13 == 1) conections to servers, where the number of data bytes
# from source (src_bytes(item 4)) is 500 times more than from server (dst_bytes(item 5))

In [30]:
split_rdd =  rdd.map(lambda x: x.split(','))
sol_rdd = split_rdd.filter(lambda x: x[13] == '1').map(lambda x: (x[1], int(x[4]), int(x[5])))

In [31]:
sol_rdd.take(5)

[('tcp', 272, 60397),
 ('tcp', 1511, 2957),
 ('tcp', 1735, 2766),
 ('tcp', 167, 19827),
 ('tcp', 145, 13236)]

In [32]:
sol_rdd = sol_rdd.filter(lambda x: x[2] > x[1]* 500 )

In [33]:
sol_rdd.collect()

[('tcp', 296, 507534),
 ('tcp', 296, 507534),
 ('tcp', 266, 507534),
 ('tcp', 296, 507534),
 ('tcp', 353, 759161),
 ('tcp', 351, 759161),
 ('tcp', 246, 866032),
 ('tcp', 317, 394616),
 ('tcp', 262, 744605),
 ('tcp', 173, 744605),
 ('tcp', 262, 744605),
 ('tcp', 255, 574784),
 ('tcp', 433, 1524348),
 ('tcp', 1794, 3851730),
 ('tcp', 0, 2072),
 ('tcp', 224, 2776333),
 ('tcp', 0, 2072),
 ('tcp', 465, 320362),
 ('tcp', 0, 2072),
 ('tcp', 0, 2072)]